In [1]:
#一定要在Python2环境下安装
!pip install turicreate

In [0]:
import turicreate as tc
import pandas as pd
import numpy as np

In [0]:
items=pd.read_csv('items.csv')
ratings=pd.read_csv('ratings.csv')
users=pd.read_csv('users.csv')

In [0]:
ratings.drop(['Unnamed: 0'],axis=1,inplace=True)

In [0]:
users.drop(['Unnamed: 0'],axis=1,inplace=True)

In [0]:
items.drop(['Unnamed: 0'],axis=1,inplace=True)

In [0]:
ratings.groupby('user_id')['movie_id'].count().to_frame().idxmax()

movie_id    405
dtype: int64

我们采用评价电影最多的用户（ID：405）来测试

1.归一化
2.独热编码，划分训练集ratings和测试集
3.合并训练集ratings和items，利用sklearn计算电影余弦相似度
4.合并训练集ratings和users，计算用户相似度
5.电影平均得分，用户平均打分也要算，用于最后匹配
6.最后用于预测的特征包括：用户个人平均打分，电影平均得分，与用户最接近的10%的用户对该电影的评分

In [0]:
def f1(t):
  return t.year
items['rd']=pd.to_datetime(items['release date'], errors = 'coerce').apply(f1)

In [0]:
items1=items.drop(['movie title','release date','video release date','IMDb URL'],axis=1)

In [0]:
#ratings至此完成所有处理
ratings1=ratings.drop(['unix_timestamp'],axis=1)

In [0]:
def f2(sex):
  if sex=='M':
    sex=1
  else:
    sex=0
    
  return sex
  
users['sex']=users['sex'].apply(f2)

In [0]:
users1=users.drop(['zip_code'],axis=1)

In [0]:
users1num=users1.select_dtypes(include='number')
users1str=pd.get_dummies(users1.drop(users1num.columns,axis=1))

In [0]:
from sklearn import preprocessing
mms=preprocessing.MinMaxScaler()

In [0]:
#item归一化
items1['rd']=mms.fit_transform(items1.rd.values.reshape(-1,1))

In [0]:
#item有一个rd为空，填充即可
items1=items1.fillna(axis=1,method='ffill')

In [0]:
#user归一化
users1num['age']=mms.fit_transform(users1num.age.values.reshape(-1,1))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
#users至此完成处理
users2=pd.concat([users1num,users1str],axis=1)

In [0]:
rh=ratings1.head()

In [0]:
rh

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [0]:
lm=len(set(ratings1.movie_id.values))

In [0]:
for i in range(1,lm+1): #总共的电影数量
  ratings1['m'+str(i)]=0
  

In [0]:
for i in range(len(ratings1)):
  m='m'+str(ratings1.loc[i,'movie_id'])
  ratings1.loc[i,m]=ratings1.loc[i,'rating']

In [0]:
items1.to_csv('items1.csv')
users2.to_csv('users2.csv')

In [0]:
ml=['m'+str(x) for x in range(1,1683)]


In [0]:
ratings2=ratings1.drop(['movie_id','rating'],axis=1)

In [0]:
ratings3=ratings2.groupby('user_id').sum()

In [0]:
ratings3.to_csv('ratings3.csv')

In [0]:
items1=pd.read_csv('items1.csv')
users2=pd.read_csv('users2.csv')
ratings3=pd.read_csv('ratings3.csv')

In [0]:
#结合用户信息和用户打分情况，用于得到最后的用户相似度矩阵
u2r3=pd.merge(users2,ratings3,on='user_id')

In [0]:
u2r3.drop(['Unnamed: 0'],axis=1,inplace=True)

In [0]:
#用户相似度矩阵
from  sklearn.metrics.pairwise import pairwise_distances
user_similarity  = pairwise_distances(u2r3, metric='cosine')

In [25]:
#基于用户相似度的初步预测
def predict(ratings, similarity):
    mean_user_rating  = ratings.mean(axis=1)
    #We use  np.newaxis so that mean_user_rating has same format as ratings
    ratings_diff =  (ratings - mean_user_rating[:, np.newaxis])
    pred =  mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) /  np.array([np.abs(similarity).sum(axis=1)]).T
    return pred
  
  user_prediction = predict(ratings, user_similarity)

0.9840428043034449

In [0]:
r=ratings
r.rename(columns={'movie_id':'item_id'},inplace=True)


In [0]:
from sklearn.model_selection import train_test_split
r1,r2=train_test_split(r,test_size=0.2)

In [61]:
#利用turicreate进行基于项目相似度的预测


# r.drop(['Unnamed: 0','unix_timestamp'],axis=1,inplace=True)
data = tc.SFrame(r1)

m = tc.item_similarity_recommender.create(data,target="rating")
nn = m.get_similar_items()
print(nn)

m2 = tc.item_similarity_recommender.create(data, nearest_items=nn,target='rating')


Preparing data set.

Data has 80000 observations with 943 users and 1649 items.

Data prepared in: 0.095615s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Data has 80000 observations with 943 users and 1649 items.

Data prepared in: 0.096913s

Loading user-provided nearest items.

Generating candidate set for working with new users.

Finished training in 0.023361s

In [62]:
data2=tc.SFrame(r2)
m2.predict(data2)

m2.recommend()
m2.evaluate(data2)


Precision and recall summary statistics by cutoff
+--------+----------------+-----------------+
| cutoff | mean_precision |   mean_recall   |
+--------+----------------+-----------------+
|   1    | 0.435244161359 | 0.0330448003778 |
|   2    | 0.398619957537 | 0.0592603776616 |
|   3    | 0.379688605803 | 0.0829692782894 |
|   4    | 0.366242038217 |  0.105420938316 |
|   5    | 0.347558386412 |  0.121434504401 |
|   6    | 0.331564048125 |  0.137146069857 |
|   7    | 0.320746132848 |  0.15389207189  |
|   8    | 0.313826963907 |  0.169561071813 |
|   9    | 0.303373437131 |  0.182223296845 |
|   10   | 0.293099787686 |  0.193445451611 |
+--------+----------------+-----------------+
[10 rows x 3 columns]


Overall RMSE: 3.69538328159

Per User RMSE (best)
+---------+-------+---------------+
| user_id | count |      rmse     |
+---------+-------+---------------+
|   685   |   5   | 1.46554017414 |
+---------+-------+---------------+
[1 rows x 3 columns]


Per User RMSE (worst)
+-----

{'precision_recall_by_user': Columns:
 	user_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 16956
 
 Data:
 +---------+--------+----------------+-----------------+-------+
 | user_id | cutoff |   precision    |      recall     | count |
 +---------+--------+----------------+-----------------+-------+
 |    1    |   1    |      1.0       | 0.0169491525424 |   59  |
 |    1    |   2    |      1.0       | 0.0338983050847 |   59  |
 |    1    |   3    | 0.666666666667 | 0.0338983050847 |   59  |
 |    1    |   4    |      0.5       | 0.0338983050847 |   59  |
 |    1    |   5    |      0.6       | 0.0508474576271 |   59  |
 |    1    |   6    | 0.666666666667 | 0.0677966101695 |   59  |
 |    1    |   7    | 0.571428571429 | 0.0677966101695 |   59  |
 |    1    |   8    |      0.5       | 0.0677966101695 |   59  |
 |    1    |   9    | 0.444444444444 | 0.0677966101695 |   59  |
 |    1    |   10   |      0.4       | 0.0677966101695 |   59  |
 +---------+--------+